**Will Steph Curry make his team win given the conditions of that match which are basically Curry's stats of that match.**

So basically the dataset I'm using has various independent variables but I have considered only important features.

Feature to be selected are: Dates , Opponent , Minutes , Successful Shots , Total Shots , 3 Points Successfull , Total 3 Points , Successful FT , Total FT , REB , AST , BLK , STL , PTS , Type

Dependent variable/ Prediction : Result

**IMPORTING THE REQUIRED LIBRARIES**

In [989]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

**IMPORTING THE DATASET**

In [990]:
df_xlsx = pd.read_excel('Stephen Curry Stats.xlsx')
df_xlsx.to_csv('csvfile.csv', encoding='utf-8', index=False)
df = pd.read_csv('csvfile.csv')

**GETTING A PEEK IN THE DATASET**

In [991]:
df.dtypes

Dates                 object
Opponent              object
Score                 object
Minutes                int64
Successful Shots       int64
Total Shots            int64
3 Points Succesful     int64
Total 3 Points         int64
Successful FT          int64
Total FT               int64
REB                    int64
AST                    int64
BLK                    int64
STL                    int64
PF                     int64
TO                     int64
PTS                    int64
Type                  object
Result                object
Score GS               int64
Score Opponent         int64
dtype: object

In [992]:
df.head()

,Dates,Opponent,Score,Minutes,Successful Shots,Total Shots,3 Points Succesful,Total 3 Points,Successful FT,Total FT,REB,AST,BLK,STL,PF,TO,PTS,Type,Result,Score GS,Score Opponent
0,2009-10-28,HOU,L 107-108,36,7,12,0,1,0,0,2,7,0,4,2,2,14,REGULAR SEASON STATS,L,107,108
1,2009-10-30,PHX,L 101-123,39,5,9,2,3,0,0,2,4,0,1,5,3,12,REGULAR SEASON STATS,L,101,123
2,2009-11-04,MEM,W 113-105,28,3,6,1,2,0,0,5,9,0,2,4,1,7,REGULAR SEASON STATS,W,113,105
3,2009-11-06,LAC,L 90-118,22,1,5,0,1,3,4,1,3,0,0,6,0,5,REGULAR SEASON STATS,L,90,118
4,2009-11-08,SAC,L 107-120,31,4,8,1,2,0,2,4,6,0,0,4,5,9,REGULAR SEASON STATS,L,107,120


**DROPPING NOT SO USEFUL DATA**

In [993]:
df = df.drop(['Score' , 'PF' , 'TO' , 'Score Opponent' , 'Score GS' , 'Score Opponent'] , axis=1)

**SINCE TYPE COLUMN HAS NAN VALUES SO WE NEED TO REMOVE ROWS WITH NAN VALUES**

In [994]:
df['Type'].unique()

array(['REGULAR SEASON STATS', nan, 'Conference Quarterfinals',
       'Conference Semifinals', 'Conference Finals', 'NBA Finals'],
      dtype=object)

In [995]:
df.dropna(subset = ["Type"], inplace=True)

**SINCE THERE ARE ONLY 30 TEAMS IN NBA BUT WE HAVE 33 UNIQUE VALUES IN OUR OPPONENT COLUMN AND WE ONLY NEED 29 BECAUSE 30 - 1(GSW) = 29**

1. REPLACE NJ COLUMNS WITH BKN SINCE THE TEAM IS SAME ONLY THE NAME HAVE CHANGED.

2. REMOVE COLUMNS WITH EAST IN IT SINCE THERE IS NO SUCH TEAM.

3. REMOCE COLUMNS WITH MCH , LEB ALSO.

In [996]:
df['Opponent'].unique()

array(['HOU', 'PHX', 'MEM', 'LAC', 'SAC', 'MIN', 'IND', 'NY', 'MIL',
       'CLE', 'BOS', 'POR', 'DAL', 'SA', 'LAL', 'DEN', 'ORL', 'OKC', 'NJ',
       'CHI', 'DET', 'PHI', 'WSH', 'NO', 'MIA', 'CHA', 'UTAH', 'ATL',
       'TOR', 'BKN', 'EAST', 'LEB'], dtype=object)

In [997]:
df = df.replace(to_replace =['NJ'],
                 value =['BKN'])

In [998]:
df = df[df['Opponent'] != 'EAST']
df = df[df['Opponent'] != 'MCH']
df = df[df['Opponent'] != 'LEB']

In [999]:
df['Opponent'].unique()

array(['HOU', 'PHX', 'MEM', 'LAC', 'SAC', 'MIN', 'IND', 'NY', 'MIL',
       'CLE', 'BOS', 'POR', 'DAL', 'SA', 'LAL', 'DEN', 'ORL', 'OKC',
       'BKN', 'CHI', 'DET', 'PHI', 'WSH', 'NO', 'MIA', 'CHA', 'UTAH',
       'ATL', 'TOR'], dtype=object)

**CHANGING DTYPES OF SOME COLUMNS**

Dates is changed from object dtype to DateTime

Opponent to string

Type and Result to string

In [1000]:
df['Dates'] = pd.to_datetime(df['Dates'])
print(df['Dates'].dtype)

datetime64[ns]


In [1001]:
df["Opponent"] = df["Opponent"].astype("|S")
print(df['Opponent'].dtype)

|S4


In [1002]:
df["Type"] = df["Type"].astype("|S")
print(df['Type'].dtype)

|S24


In [1003]:
df["Result"] = df["Result"].astype("|S")
print(df['Result'].dtype)

|S1


**CHECKING COLUMNS WITH NEED OF LABEL ENCODING:**

1. Opponents

2. Type

3. Result

In [1004]:
lb = LabelEncoder()
df['Opponent']= lb.fit_transform(df['Opponent'])
df['Opponent'].unique()

array([ 9, 22, 13, 11, 25, 16, 10, 18, 15,  5,  2, 23,  6, 24, 12,  7, 20,
       19,  1,  4,  8, 21, 28, 17, 14,  3, 27,  0, 26])

In [1005]:
df['Type'].unique()

array([b'REGULAR SEASON STATS', b'Conference Quarterfinals',
       b'Conference Semifinals', b'Conference Finals', b'NBA Finals'],
      dtype='|S24')

In [1006]:
df['Type']= lb.fit_transform(df['Type'])
df['Type'].unique()

array([4, 1, 2, 0, 3])

In [1007]:
df['Result'].unique()

array([b'L', b'W'], dtype='|S1')

In [1008]:
df['Result']= lb.fit_transform(df['Result'])
df['Result'].unique()

array([0, 1])

**CHECKING THE NEW DATA FORMED AFTER LABEL ENCODING AND CLEANING**

In [1009]:
df.head(5)

,Dates,Opponent,Minutes,Successful Shots,Total Shots,3 Points Succesful,Total 3 Points,Successful FT,Total FT,REB,AST,BLK,STL,PTS,Type,Result
0,2009-10-28,9,36,7,12,0,1,0,0,2,7,0,4,14,4,0
1,2009-10-30,22,39,5,9,2,3,0,0,2,4,0,1,12,4,0
2,2009-11-04,13,28,3,6,1,2,0,0,5,9,0,2,7,4,1
3,2009-11-06,11,22,1,5,0,1,3,4,1,3,0,0,5,4,0
4,2009-11-08,25,31,4,8,1,2,0,2,4,6,0,0,9,4,0


In [1010]:
df.dtypes

Dates                 datetime64[ns]
Opponent                       int64
Minutes                        int64
Successful Shots               int64
Total Shots                    int64
3 Points Succesful             int64
Total 3 Points                 int64
Successful FT                  int64
Total FT                       int64
REB                            int64
AST                            int64
BLK                            int64
STL                            int64
PTS                            int64
Type                           int64
Result                         int64
dtype: object

**CHANGING DATETIME OF DATE COLUMN TO ONLY YEAR(FLOAT)**

In [1011]:
df['Dates'] = pd.DatetimeIndex(df['Dates']).year

In [1012]:
df.dtypes

Dates                 int64
Opponent              int64
Minutes               int64
Successful Shots      int64
Total Shots           int64
3 Points Succesful    int64
Total 3 Points        int64
Successful FT         int64
Total FT              int64
REB                   int64
AST                   int64
BLK                   int64
STL                   int64
PTS                   int64
Type                  int64
Result                int64
dtype: object

**SPLITTING THE DATASET INTO TRAINING AND TESTING**

In [1013]:
x = df.iloc[: , :-1].values
y = df.iloc[: , -1].values

In [1014]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 0)

**FEATURE SCALING OUR DATA**

In [1015]:
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.fit_transform(x_test)

**TRAINING OUR DATASET**

**MODEL 1 : NAIVE BAYES**

In [1016]:
model1 = GaussianNB()
model1.fit(x_train , y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [1017]:
y_pred = model1.predict(x_test)

In [1018]:
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[41 37]
 [31 99]]


0.6730769230769231

**MODEL 2 : SVC NON LINEAR**

In [1019]:
model2 = SVC(kernel = 'rbf', random_state = 0)
model2.fit(x_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=0, shrinking=True, tol=0.001,
    verbose=False)

In [1020]:
y_pred = model2.predict(x_test)

In [1021]:
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[ 37  41]
 [ 16 114]]


0.7259615384615384

**MODEL 3 : SVC LINEAR**

In [1022]:
model3 = SVC(kernel = 'linear', random_state = 0)
model3.fit(x_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=0, shrinking=True, tol=0.001,
    verbose=False)

In [1023]:
y_pred = model3.predict(x_test)

In [1024]:
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[ 44  34]
 [ 24 106]]


0.7211538461538461

**MODEL 4 : RANDOM FOREST CLASSIFIER**

In [1025]:
model4 = RandomForestClassifier(n_estimators = 5, criterion = 'entropy', random_state = 0)
model4.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=5,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [1026]:
y_pred = model4.predict(x_test)

In [1027]:
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[36 42]
 [31 99]]


0.6490384615384616

**MODEL 5 : LOGISTIC REGRESSION**

In [1028]:
model5 = LogisticRegression(random_state= 0)
model5.fit(x_train , y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [1029]:
y_pred = model5.predict(x_test)

In [1030]:
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[ 44  34]
 [ 23 107]]


0.7259615384615384

**MODEL 6 : DECISION TREE**

In [1031]:
model6 = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
model6.fit(x_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=0, splitter='best')

In [1032]:
y_pred = model6.predict(x_test)

In [1033]:
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[35 43]
 [37 93]]


0.6153846153846154

**CONCLUSION**

SINCE IT IS VISIBLE THAT OUR MODEL OF LOGISTIC REGRESSION HAD THE BEST ACCURACY SO THAT'S OUR FINAL MODEL AFTER THE COMPLETE EVALUATION. ALSO RATHER THAT SPLITTING OUR DATASET TO TRAIN AND TEST, I'M TRAINING OUR WHOLE DATASET WITH LOGISTIC REGRESSION. IT WILL BE MORE BENEFICIAL.

In [1034]:
modelfinal = LogisticRegression(random_state= 0)
modelfinal.fit(sc.fit_transform(x) , y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [1035]:
y_pred = modelfinal.predict(x_test)

In [1036]:
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[ 50  28]
 [ 24 106]]


0.75

**FUNCTION FOR MAKING PREDICTIONS EASIER**

**MAKING PREDICTIONS**

In [1037]:
def PredictingGames():
  inputs = [[]]
  NBAYear = int(input("Enter the year"))
  inputs[0].append(NBAYear)
  print(inputs[0][-1])

  opponent = input("Enter the team :")
  if opponent == 'HOU':
    inputs[0].append(9)
  elif opponent == 'PHX':
    inputs[0].append(22)
  elif opponent == 'MEM':
    inputs[0].append(13)
  elif opponent == 'LAC':
    inputs[0].append(11)
  elif opponent == 'SAC':
    inputs[0].append(25)
  elif opponent == 'MIN':
    inputs[0].append(16)
  elif opponent == 'IND':
    inputs[0].append(10)
  elif opponent == 'NY':
    inputs[0].append(18)
  elif opponent == 'MIL':
    inputs[0].append(15)
  elif opponent == 'CLE':
    inputs[0].append(5)
  elif opponent == 'BOS':
    inputs[0].append(2)
  elif opponent == 'POR':
    inputs[0].append(23)
  elif opponent == 'DAL':
    inputs[0].append(6)
  elif opponent == 'SA':
    inputs[0].append(24)
  elif opponent == 'LAL':
    inputs[0].append(12)
  elif opponent == 'DEN':
    inputs[0].append(7)
  elif opponent == 'ORL':
    inputs[0].append(20)
  elif opponent == 'OKC':
    inputs[0].append(19)
  elif opponent == 'CHI':
    inputs[0].append(4)
  elif opponent == 'DET':
    inputs[0].append(8)
  elif opponent == 'PHI':
    inputs[0].append(21)
  elif opponent == 'WSH':
    inputs[0].append(28)
  elif opponent == 'NO':
    inputs[0].append(17)
  elif opponent == 'MIA':
    inputs[0].append(14)
  elif opponent == 'CHA':
    inputs[0].append(3)
  elif opponent == 'UTAH':
    inputs[0].append(27)
  elif opponent == 'ATL':
    inputs[0].append(0)
  elif opponent == 'TOR':
    inputs[0].append(26)
  elif opponent == 'BKN':
    inputs[0].append(1)
    print(opponent , inputs[0][-1])
  else:
    print("NO SUCH TEAM")
    return
  

  minutes = int(input("Enter the time in minutes curry played :"))
  inputs[0].append(minutes)
  print(inputs[0][-1])

  SuccessfulShots = int(input("Enter the total successful shots curry was able to make"))
  inputs[0].append(SuccessfulShots)
  print(inputs[0][-1])

  TotalShots = int(input("Enter the total shots curry took :"))
  inputs[0].append(TotalShots)
  print(inputs[0][-1])

  ThreePointsSuccessfull = int(input("Enter the total successful three pointer shots curry made :"))
  inputs[0].append(ThreePointsSuccessfull)
  print(inputs[0][-1])

  TotalThreePoints = int(input("Enter the total three pointer shots curry took :"))
  inputs[0].append(TotalThreePoints)
  print(inputs[0][-1])

  FreeThrowSuccessfull = int(input("Enter the total successful Free Throw shots curry made :"))
  inputs[0].append(FreeThrowSuccessfull)
  print(inputs[0][-1])

  TotalFreeThrow = int(input("Enter the total Free Throw curry took :"))
  inputs[0].append(TotalFreeThrow)
  print(inputs[0][-1])

  Rebound = int(input("Total rebounds curry made : "))
  inputs[0].append(Rebound)
  print(inputs[0][-1])

  Assist = int(input("Total Assists curry made : "))
  inputs[0].append(Assist)
  print(inputs[0][-1])

  Block = int(input("Total Block curry made : "))
  inputs[0].append(Block)
  print(inputs[0][-1])

  Steal = int(input("Total Steal curry made : "))
  inputs[0].append(Steal)
  print(inputs[0][-1])

  Points = int(input("Total Points curry had : "))
  inputs[0].append(Points)
  print(inputs[0][-1])

  Types = input("What was the match type : (0 for Conference Finals ; 1 for Conference QuarterFinals ; 2 for Conference SemiFinals ; 3 for NBA Finals ; 4 for Regulas Season)")
  inputs[0].append(Types)

  inputs = np.array(inputs)
  inputs = inputs.astype(np.float64)
  inputs = sc.fit_transform(inputs)
  result = modelfinal.predict((inputs))
  if result == 1:
    print("Golden State Warriors WON!!")
  else:
    print("Golden State Warriors LOST!!")

  

PredictingGames()

Enter the year2021
2021
Enter the team :CHI
Enter the time in minutes curry played :34
34
Enter the total successful shots curry was able to make13
13
Enter the total shots curry took :19
19
Enter the total successful three pointer shots curry made :13
13
Enter the total three pointer shots curry took :19
19
Enter the total successful Free Throw shots curry made :0
0
Enter the total Free Throw curry took :0
0
Total rebounds curry made : 3
3
Total Assists curry made : 9
9
Total Block curry made : 1
1
Total Steal curry made : 3
3
Total Points curry had : 41
41
What was the match type : (0 for Conference Finals ; 1 for Conference QuarterFinals ; 2 for Conference SemiFinals ; 3 for NBA Finals ; 4 for Regulas Season)3
Golden State Warriors WON!!
